In [181]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from tqdm import tqdm
import catboost as cbt
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
%matplotlib inline
pd.set_option('display.max_columns', None)

In [182]:
#!pip install catboost==0.15

In [183]:
df_train = pd.read_csv('/home/kesci/input/smart_edu7557/exam_score.csv')
df_train = df_train[(df_train['score']!=0)].reset_index(drop=True)
# tmp1 = df_train.groupby(by=['student_id','course'], as_index=False)['score'].agg({'mean_score':np.mean, 'median_score':np.median, 'std_score':np.std,'max_score':np.max,'min_score':np.min})
# df_train=df_train.merge(tmp1, on=['student_id','course'], how='left')
# col_drop=[i for i in df_train.index if df_train.loc[i]['std_score']>17 and  df_train.loc[i]['score']==df_train.loc[i]['min_score']]
# print(col_drop)
# df_train=df_train.drop(col_drop)
# #将列去掉
# df_train=df_train.drop(['mean_score', 'median_score', 'std_score','max_score','min_score'],axis=1)
print(df_train.shape)

(303899, 4)


In [184]:
#!killall python


In [185]:

# features_score_direct=extract_relevant_features(df_train ,column_id='student_id' and 'course',column_sort='exam_id')
# print(features_score_direct.head())
# process_index = lambda x: list(x[((x >= (np.percentile(x,25) - 1.5*(np.percentile(x,75)-np.percentile(x,25)))) & 
# (x <= (np.percentile(x,75) + 1.5*(np.percentile(x,75)-np.percentile(x,25)))))].index)
# tmp_process = df_train.groupby(by=['student_id','course'], as_index=False)['score'].agg({'process_index':process_index})

In [186]:
# index_list = []
# for i in tmp_process['process_index']:
#     index_list = index_list+i

In [187]:
# df_train = df_train.iloc[index_list]
# df_train = df_train.reset_index(drop=True)

In [188]:
len(df_train)

303899

In [189]:
df_test = pd.read_csv('/home/kesci/input/smart_edu7557/submission_s2.csv')
df_test.rename(columns={'pred':'score'},inplace = True)
course_class = pd.read_csv('/home/kesci/input/smart_edu7557/course.csv')
student = pd.read_csv('/home/kesci/input/smart_edu7557/student.csv')
all_know = pd.read_csv('/home/kesci/input/smart_edu7557/all_knowledge.csv')
df_all = df_train.append(df_test)
df_all = df_all.merge(course_class, on='course', how='left')
df_all = df_all.merge(student, on='student_id', how='left')

In [190]:
from sklearn.metrics.pairwise import cosine_distances
course1_exam = pd.read_csv('/home/kesci/input/smart_edu7557/course1_exams.csv')
course2_exam = pd.read_csv('/home/kesci/input/smart_edu7557/course2_exams.csv')
course3_exam = pd.read_csv('/home/kesci/input/smart_edu7557/course3_exams.csv')
course4_exam = pd.read_csv('/home/kesci/input/smart_edu7557/course4_exams.csv')
course5_exam = pd.read_csv('/home/kesci/input/smart_edu7557/course5_exams.csv')
course6_exam = pd.read_csv('/home/kesci/input/smart_edu7557/course6_exams.csv')
course7_exam = pd.read_csv('/home/kesci/input/smart_edu7557/course7_exams.csv')
course8_exam = pd.read_csv('/home/kesci/input/smart_edu7557/course8_exams.csv')
#col_c1 = [i for i in course1_exam.columns if i not in ['course','exam_id']]
tmp4=1
for i in [course1_exam,course2_exam,course3_exam,course4_exam,course5_exam,course6_exam,course7_exam,course8_exam]:
    name = i
    col_c1 = [i for i in name.columns if i not in ['course','exam_id']]
    name['course'] ='course'+str(tmp4)
    tmp2 =np.array(all_know.loc[all_know['course'] == ('course'+str(tmp4)),:]['complexity'])
    tmp = name[col_c1]
    tmp3 =np.dot(tmp.values,tmp2)
    name['hard'] = tmp3
    name['hard_inverse'] = name['hard'].apply(lambda x:1/(x+1e-10))
    tmp4 = tmp4+1
    
    np_tmp = name[col_c1].values
    np_tmp= np_tmp.astype(np.bool)
    np_tmp2 = np.sum(np_tmp,axis=1)
    np_tmp = np.sum(np_tmp,axis=1)/len(col_c1)
    np_tmp = (np_tmp-np_tmp.min())/(np_tmp.max()-np_tmp.min())
    name['ration_know'] = np_tmp
    name['number_know'] = np_tmp2
    #添加每个知识点的均分
    name['know_mean']=[100/i for i in np_tmp2]
    
for i in [course1_exam,course2_exam,course3_exam,course4_exam,course5_exam,course6_exam,course7_exam,course8_exam]:
    name = i
    col_c1 = [i for i in name.columns if i not in ['course','exam_id','hard','hard_inverse','ration_know','number_know']]
    pd_ = name[col_c1]
    e = 1-cosine_distances(pd_)
    e = e -np.diag([1] * len(name))
    inde = np.argmax(e,axis=1)
    pd_['inde'] = inde
    name['sim_exam_id'] = pd_['inde'].apply(lambda x:i.loc[x,'exam_id'])
    
for key in all_know.groupby(['course','section'])['knowledge_point'].groups.keys():
    course = key[0]
    section = key[1]
    if course == 'course1':
        s0_inde = all_know.groupby(['course','section'])['knowledge_point'].groups[key]
        tmp = course1_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course1_exam[section] = tmp.apply(np.sum,axis = 1)
    elif course == 'course2':
        s0_inde = all_know.groupby(['course','section'])['knowledge_point'].groups[key]
        tmp = course2_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course2_exam[section] = tmp.apply(np.sum,axis = 1)
    elif course == 'course3':
        s0_inde = all_know.groupby(['course','section'])['knowledge_point'].groups[key]
        tmp = course3_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course3_exam[section] = tmp.apply(np.sum,axis = 1)
    elif course == 'course4':
        s0_inde = all_know.groupby(['course','section'])['knowledge_point'].groups[key]
        tmp = course4_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course4_exam[section] = tmp.apply(np.sum,axis = 1)
    elif course == 'course5':
        s0_inde = all_know.groupby(['course','section'])['knowledge_point'].groups[key]
        tmp = course5_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course5_exam[section] = tmp.apply(np.sum,axis = 1)
    elif course == 'course6':
        s0_inde = all_know.groupby(['course','section'])['knowledge_point'].groups[key]
        tmp = course6_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course6_exam[section] = tmp.apply(np.sum,axis = 1)
    elif course == 'course7':
        s0_inde = all_know.groupby(['course','section'])['knowledge_point'].groups[key]
        tmp = course7_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course7_exam[section] = tmp.apply(np.sum,axis = 1)
    elif course == 'course8':
        s0_inde = all_know.groupby(['course','section'])['knowledge_point'].groups[key]
        tmp = course8_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course8_exam[section] = tmp.apply(np.sum,axis = 1)
for key in all_know.groupby(['course','category'])['knowledge_point'].groups.keys():
    course = key[0]
    section = key[1]
    if course == 'course1':
        s0_inde = all_know.groupby(['course','category'])['knowledge_point'].groups[key]
        tmp = course1_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course1_exam[section] = tmp.apply(np.sum,axis = 1)
    elif course == 'course2':
        s0_inde = all_know.groupby(['course','category'])['knowledge_point'].groups[key]
        tmp = course2_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course2_exam[section] = tmp.apply(np.sum,axis = 1)
    elif course == 'course3':
        s0_inde = all_know.groupby(['course','category'])['knowledge_point'].groups[key]
        tmp = course3_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course3_exam[section] = tmp.apply(np.sum,axis = 1)
    elif course == 'course4':
        s0_inde = all_know.groupby(['course','category'])['knowledge_point'].groups[key]
        tmp = course4_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course4_exam[section] = tmp.apply(np.sum,axis = 1)
    elif course == 'course5':
        s0_inde = all_know.groupby(['course','category'])['knowledge_point'].groups[key]
        tmp = course5_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course5_exam[section] = tmp.apply(np.sum,axis = 1)
    elif course == 'course6':
        s0_inde = all_know.groupby(['course','category'])['knowledge_point'].groups[key]
        tmp = course6_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course6_exam[section] = tmp.apply(np.sum,axis = 1)
    elif course == 'course7':
        s0_inde = all_know.groupby(['course','category'])['knowledge_point'].groups[key]
        tmp = course7_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course7_exam[section] = tmp.apply(np.sum,axis = 1)
    elif course == 'course8':
        s0_inde = all_know.groupby(['course','category'])['knowledge_point'].groups[key]
        tmp = course8_exam[list(all_know.iloc[s0_inde]['knowledge_point'])]
        course8_exam[section] = tmp.apply(np.sum,axis = 1)     
for i in [course1_exam,course2_exam,course3_exam,course4_exam,course5_exam,course6_exam,course7_exam,course8_exam]:
    name = i
    col_c1 = [i for i in name.columns if 'S:' in i ]
    pd_ = name[col_c1]
    e = 1-cosine_distances(pd_)
    e = e -np.diag([1] * len(name))
    inde = np.argmax(e,axis=1)
    pd_['inde'] = inde
    name['sectionsim_exam_id'] = pd_['inde'].apply(lambda x:i.loc[x,'exam_id'])
for i in [course1_exam,course2_exam,course3_exam,course4_exam,course5_exam,course6_exam,course7_exam,course8_exam]:
    name = i
    col_c1 = [i for i in name.columns if 'C:' in i ]
    pd_ = name[col_c1]
    e = 1-cosine_distances(pd_)
    e = e -np.diag([1] * len(name))
    inde = np.argmax(e,axis=1)
    pd_['inde'] = inde
    name['catsim_exam_id'] = pd_['inde'].apply(lambda x:i.loc[x,'exam_id'])
cluster_count =1    
for name in [course1_exam,course2_exam,course3_exam,course4_exam,course5_exam,course6_exam,course7_exam,course8_exam]:
    col_K = [i for i in name.columns if 'K:' in i]
    kmeans = KMeans(n_clusters=20, random_state=0).fit(name[col_K])
    name['cluster'] = kmeans.labels_
    name['cluster']= name['cluster'].apply(lambda x:'c'+str(cluster_count)+str(x))
    cluster_count = cluster_count+1
for name in [course1_exam,course2_exam,course3_exam,course4_exam,course5_exam,course6_exam,course7_exam,course8_exam]:
    col_K = [i for i in name.columns if 'K:' in i]
    kmeans = KMeans(n_clusters=30, random_state=0).fit(name[col_K])
    name['cluster_30'] = kmeans.labels_
    name['cluster_30']= name['cluster_30'].apply(lambda x:'c'+str(cluster_count)+str(x))
    cluster_count = cluster_count+1
for name in [course1_exam,course2_exam,course3_exam,course4_exam,course5_exam,course6_exam,course7_exam,course8_exam]:
    col_K = [i for i in name.columns if 'K:' in i]
    kmeans = KMeans(n_clusters=40, random_state=0).fit(name[col_K])
    name['cluster_40'] = kmeans.labels_
    name['cluster_40']= name['cluster_40'].apply(lambda x:'c'+str(cluster_count)+str(x))
    cluster_count = cluster_count+1
for name in [course1_exam,course2_exam,course3_exam,course4_exam,course5_exam,course6_exam,course7_exam,course8_exam]:
    col_K = [i for i in name.columns if 'K:' in i]
    kmeans = KMeans(n_clusters=50, random_state=0).fit(name[col_K])
    name['cluster_50'] = kmeans.labels_
    name['cluster_50']= name['cluster_50'].apply(lambda x:'c'+str(cluster_count)+str(x))
    cluster_count = cluster_count+1
course_exam = course1_exam.append(course2_exam)
course_exam = course_exam.append(course3_exam)
course_exam = course_exam.append(course4_exam)
course_exam = course_exam.append(course5_exam)
course_exam = course_exam.append(course6_exam)
course_exam = course_exam.append(course7_exam)
course_exam = course_exam.append(course8_exam)
course_exam.fillna(0,inplace = True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [191]:
df_all.head()
#对每个考生的每个科目的成绩的变动制造特征

    

,student_id,course,exam_id,score,course_class,gender
0,230748,course1,XBQWgzwe,71.0,course_class1,0
1,186851,course1,XBQWgzwe,75.0,course_class1,0
2,478370,course1,XBQWgzwe,86.0,course_class1,0
3,692328,course1,XBQWgzwe,81.0,course_class1,1
4,509128,course1,XBQWgzwe,75.0,course_class1,0


In [192]:
from sklearn.decomposition import PCA
col=[i for i in course_exam.columns if i not in ['cluster_50','cluster_40','cluster_30','cluster','course','exam_id','hard','hard_inverse','ration_know','number_know','sim_exam_id','sectionsim_exam_id','catsim_exam_id']]
print(len(col))
col_K = [i for i in col if 'K:' in i]
col_S = [i for i in col if 'S:' in i]
col_C = [i for i in col if 'C:' in i]
x_K=course_exam[col_K].values
x_S=course_exam[col_S].values
x_C=course_exam[col_C].values
#print(type(x))
pca=PCA(n_components=20)     #加载PCA算法，设置降维后主成分数目为5
reduced_x_K=pca.fit_transform(x_K)#对样本进行降维，x为特征组成的矩阵
#print(reduced_x[:,0].shape)
#将特征添加到course_exam
course_exam['k_feature_1']=reduced_x_K[:,0]
course_exam['k_feature_2']=reduced_x_K[:,1]
course_exam['k_feature_3']=reduced_x_K[:,2]
course_exam['k_feature_4']=reduced_x_K[:,3]
course_exam['k_feature_5']=reduced_x_K[:,4]
course_exam['k_feature_6']=reduced_x_K[:,5]
course_exam['k_feature_7']=reduced_x_K[:,6]
course_exam['k_feature_8']=reduced_x_K[:,7]
course_exam['k_feature_9']=reduced_x_K[:,8]
course_exam['k_feature_10']=reduced_x_K[:,9]
course_exam['k_feature_11']=reduced_x_K[:,10]
course_exam['k_feature_12']=reduced_x_K[:,11]
course_exam['k_feature_13']=reduced_x_K[:,12]
course_exam['k_feature_14']=reduced_x_K[:,13]
course_exam['k_feature_15']=reduced_x_K[:,14]
course_exam['k_feature_16']=reduced_x_K[:,15]
course_exam['k_feature_17']=reduced_x_K[:,16]
course_exam['k_feature_18']=reduced_x_K[:,17]
course_exam['k_feature_19']=reduced_x_K[:,18]
course_exam['k_feature_20']=reduced_x_K[:,19]

reduced_x_C=pca.fit_transform(x_C)#对样本进行降维，x为特征组成的矩阵
#print(reduced_x[:,0].shape)
#将特征添加到course_exam
course_exam['C_feature_1']=reduced_x_C[:,0]
course_exam['C_feature_2']=reduced_x_C[:,1]
course_exam['C_feature_3']=reduced_x_C[:,2]
course_exam['C_feature_4']=reduced_x_C[:,3]
course_exam['C_feature_5']=reduced_x_C[:,4]
course_exam['C_feature_6']=reduced_x_C[:,5]
course_exam['C_feature_7']=reduced_x_C[:,6]
course_exam['C_feature_8']=reduced_x_C[:,7]
course_exam['C_feature_9']=reduced_x_C[:,8]
course_exam['C_feature_10']=reduced_x_C[:,9]
course_exam['C_feature_11']=reduced_x_C[:,10]
course_exam['C_feature_12']=reduced_x_C[:,11]
course_exam['C_feature_13']=reduced_x_C[:,12]
course_exam['C_feature_14']=reduced_x_C[:,13]
course_exam['C_feature_15']=reduced_x_C[:,14]
course_exam['C_feature_16']=reduced_x_C[:,15]
course_exam['C_feature_17']=reduced_x_C[:,16]
course_exam['C_feature_18']=reduced_x_C[:,17]
course_exam['C_feature_19']=reduced_x_C[:,18]
course_exam['C_feature_20']=reduced_x_C[:,19]

reduced_x_S=pca.fit_transform(x_S)#对样本进行降维，x为特征组成的矩阵
#print(reduced_x[:,0].shape)
#将特征添加到course_exam
course_exam['S_feature_1']=reduced_x_S[:,0]
course_exam['S_feature_2']=reduced_x_S[:,1]
course_exam['S_feature_3']=reduced_x_S[:,2]
course_exam['S_feature_4']=reduced_x_S[:,3]
course_exam['S_feature_5']=reduced_x_S[:,4]
course_exam['S_feature_6']=reduced_x_S[:,5]
course_exam['S_feature_7']=reduced_x_S[:,6]
course_exam['S_feature_8']=reduced_x_S[:,7]
course_exam['S_feature_9']=reduced_x_S[:,8]
course_exam['S_feature_10']=reduced_x_S[:,9]
course_exam['S_feature_11']=reduced_x_S[:,10]
course_exam['S_feature_12']=reduced_x_S[:,11]
course_exam['S_feature_13']=reduced_x_S[:,12]
course_exam['S_feature_14']=reduced_x_S[:,13]
course_exam['S_feature_15']=reduced_x_S[:,14]
course_exam['S_feature_16']=reduced_x_S[:,15]
course_exam['S_feature_17']=reduced_x_S[:,16]
course_exam['S_feature_18']=reduced_x_S[:,17]
course_exam['S_feature_19']=reduced_x_S[:,18]
course_exam['S_feature_20']=reduced_x_S[:,19]

sub_course_exam = course_exam[['cluster_50','cluster_40','cluster_30','cluster','course','exam_id','hard','hard_inverse','ration_know','number_know','sim_exam_id','sectionsim_exam_id','catsim_exam_id']]
sub_course_exam_K = course_exam[['course','exam_id','k_feature_1','k_feature_2','k_feature_3','k_feature_4','k_feature_5','k_feature_6','k_feature_7','k_feature_8','k_feature_9','k_feature_10','k_feature_11','k_feature_12','k_feature_13','k_feature_14','k_feature_15','k_feature_16','k_feature_17','k_feature_18','k_feature_19','k_feature_20']]
sub_course_exam_S = course_exam[['course','exam_id','S_feature_1','S_feature_2','S_feature_3','S_feature_4','S_feature_5','S_feature_6','S_feature_7','S_feature_8','S_feature_9','S_feature_10','S_feature_11','S_feature_12','S_feature_13','S_feature_14','S_feature_15','S_feature_16','S_feature_17','S_feature_18','S_feature_19','S_feature_20']]
sub_course_exam_C = course_exam[['course','exam_id','C_feature_1','C_feature_2','C_feature_3','C_feature_4','C_feature_5','C_feature_6','C_feature_7','C_feature_8','C_feature_9','C_feature_10','C_feature_11','C_feature_12','C_feature_13','C_feature_14','C_feature_15','C_feature_16','C_feature_17','C_feature_18','C_feature_19','C_feature_20']]
df_all=df_all.merge(sub_course_exam, on=['exam_id','course'], how='left')
df_all=df_all.merge(sub_course_exam_K, on=['exam_id','course'], how='left')
df_all=df_all.merge(sub_course_exam_S, on=['exam_id','course'], how='left')
df_all=df_all.merge(sub_course_exam_C, on=['exam_id','course'], how='left')

482


In [193]:
course_exam.head()

,C:0,C:1,C:10,C:11,C:12,C:13,C:14,C:15,C:16,C:17,C:18,C:19,C:2,C:20,C:21,C:22,C:23,C:24,C:25,C:26,C:27,C:28,C:29,C:3,C:30,C:31,C:32,C:33,C:34,C:4,C:5,C:6,C:7,C:8,C:9,K:0,K:1,K:10,K:100,K:101,K:102,K:103,K:104,K:105,K:106,K:107,K:108,K:109,K:11,K:110,K:111,K:112,K:113,K:114,K:115,K:116,K:117,K:118,K:119,K:12,K:120,K:121,K:122,K:123,K:124,K:125,K:126,K:127,K:128,K:129,K:13,K:130,K:131,K:132,K:133,K:134,K:135,K:136,K:137,K:138,K:139,K:14,K:140,K:141,K:142,K:143,K:144,K:145,K:146,K:147,K:148,K:149,K:15,K:150,K:151,K:152,K:153,K:154,K:155,K:156,K:157,K:158,K:159,K:16,K:160,K:161,K:162,K:163,K:164,K:165,K:166,K:167,K:168,K:169,K:17,K:170,K:171,K:172,K:173,K:174,K:175,K:176,K:177,K:178,K:179,K:18,K:180,K:181,K:182,K:183,K:184,K:185,K:186,K:187,K:188,K:189,K:19,K:190,K:191,K:192,K:193,K:194,K:195,K:196,K:197,K:198,K:199,K:2,K:20,K:200,K:201,K:202,K:203,K:204,K:205,K:206,K:207,K:208,K:209,K:21,K:210,K:211,K:212,K:213,K:214,K:215,K:216,K:217,K:218,K:219,K:22,K:220,K:221,K:222,K:223,K:224,K:225,K:226,K:227,K:228,K:229,K:23,K:230,K:231,K:232,K:233,K:234,K:235,K:236,K:237,K:238,K:239,K:24,K:240,K:241,K:242,K:243,K:244,K:245,K:246,K:247,K:248,K:249,K:25,K:250,K:251,K:252,K:253,K:254,K:255,K:256,K:257,K:258,K:259,K:26,K:260,K:261,K:262,K:263,K:264,K:265,K:266,K:267,K:268,K:269,K:27,K:270,K:271,K:272,K:273,K:274,K:275,K:276,K:277,K:278,K:279,K:28,K:280,K:281,K:282,K:283,K:284,K:285,K:286,K:287,K:288,K:289,K:29,K:290,K:291,K:292,K:293,K:294,K:295,K:296,K:297,K:298,K:299,K:3,K:30,K:300,K:301,K:302,K:303,K:304,K:305,K:306,K:307,K:308,K:309,K:31,K:310,K:311,K:312,K:313,K:314,K:315,K:316,K:317,K:318,K:319,K:32,K:320,K:321,K:322,K:323,K:324,K:325,K:326,K:327,K:328,K:329,K:33,K:330,K:331,K:332,K:333,K:334,K:335,K:336,K:337,K:338,K:339,K:34,K:340,K:341,K:342,K:343,K:344,K:345,K:346,K:347,K:348,K:349,K:35,K:350,K:351,K:352,K:353,K:354,K:355,K:356,K:357,K:358,K:359,K:36,K:360,K:361,K:362,K:363,K:364,K:365,K:366,K:367,K:37,K:38,K:39,K:4,K:40,K:41,K:42,K:43,K:44,K:45,K:46,K:47,K:48,K:49,K:5,K:50,K:51,K:52,K:53,K:54,K:55,K:56,K:57,K:58,K:59,K:6,K:60,K:61,K:62,K:63,K:64,K:65,K:66,K:67,K:68,K:69,K:7,K:70,K:71,K:72,K:73,K:74,K:75,K:76,K:77,K:78,K:79,K:8,K:80,K:81,K:82,K:83,K:84,K:85,K:86,K:87,K:88,K:89,K:9,K:90,K:91,K:92,K:93,K:94,K:95,K:96,K:97,K:98,K:99,S:0,S:1,S:10,S:11,S:12,S:13,S:14,S:15,S:16,S:17,S:18,S:19,S:2,S:20,S:21,S:22,S:23,S:24,S:25,S:26,S:27,S:28,S:29,S:3,S:30,S:31,S:32,S:33,S:34,S:35,S:36,S:37,S:38,S:39,S:4,S:40,S:41,S:42,S:43,S:44,S:45,S:46,S:47,S:48,S:49,S:5,S:50,S:51,S:52,S:53,S:54,S:55,S:56,S:57,S:58,S:59,S:6,S:60,S:61,S:62,S:63,S:64,S:65,S:66,S:67,S:68,S:69,S:7,S:70,S:71,S:72,S:73,S:74,S:75,S:76,S:77,S:8,S:9,catsim_exam_id,cluster,cluster_30,cluster_40,cluster_50,course,exam_id,hard,hard_inverse,know_mean,number_know,ration_know,sectionsim_exam_id,sim_exam_id,k_feature_1,k_feature_2,k_feature_3,k_feature_4,k_feature_5,k_feature_6,k_feature_7,k_feature_8,k_feature_9,k_feature_10,k_feature_11,k_feature_12,k_feature_13,k_feature_14,k_feature_15,k_feature_16,k_feature_17,k_feature_18,k_feature_19,k_feature_20,C_feature_1,C_feature_2,C_feature_3,C_feature_4,C_feature_5,C_feature_6,C_feature_7,C_feature_8,C_feature_9,C_feature_10,C_feature_11,C_feature_12,C_feature_13,C_feature_14,C_feature_15,C_feature_16,C_feature_17,C_feature_18,C_feature_19,C_feature_20,S_feature_1,S_feature_2,S_feature_3,S_feature_4,S_feature_5,S_feature_6,S_feature_7,S_feature_8,S_feature_9,S_feature_10,S_feature_11,S_feature_12,S_feature_13,S_feature_14,S_feature_15,S_feature_16,S_feature_17,S_feature_18,S_feature_19,S_feature_20
0,100,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,14,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [194]:
df_all.head()

,student_id,course,exam_id,score,course_class,gender,cluster_50,cluster_40,cluster_30,cluster,hard,hard_inverse,ration_know,number_know,sim_exam_id,sectionsim_exam_id,catsim_exam_id,k_feature_1,k_feature_2,k_feature_3,k_feature_4,k_feature_5,k_feature_6,k_feature_7,k_feature_8,k_feature_9,k_feature_10,k_feature_11,k_feature_12,k_feature_13,k_feature_14,k_feature_15,k_feature_16,k_feature_17,k_feature_18,k_feature_19,k_feature_20,S_feature_1,S_feature_2,S_feature_3,S_feature_4,S_feature_5,S_feature_6,S_feature_7,S_feature_8,S_feature_9,S_feature_10,S_feature_11,S_feature_12,S_feature_13,S_feature_14,S_feature_15,S_feature_16,S_feature_17,S_feature_18,S_feature_19,S_feature_20,C_feature_1,C_feature_2,C_feature_3,C_feature_4,C_feature_5,C_feature_6,C_feature_7,C_feature_8,C_feature_9,C_feature_10,C_feature_11,C_feature_12,C_feature_13,C_feature_14,C_feature_15,C_feature_16,C_feature_17,C_feature_18,C_feature_19,C_feature_20
0,230748,course1,XBQWgzwe,71.0,course_class1,0,c259,c1730,c93,c14,237,0.004219,0.047619,5,xqbAlQCz,xqbAlQCz,Y7dW6m3f,31.285633,-1.629811,5.932618,-9.677644,7.299567,-4.862344,6.169012,3.902276,3.292172,-0.99198,12.841719,10.642561,-0.6734,-3.290808,4.086778,2.139487,-1.512143,-2.402073,-0.272539,-1.267784,65.90831,10.235587,8.739786,1.621543,7.783871,11.785037,3.029331,11.168388,-9.006072,36.992494,19.882085,-12.459994,31.111172,-4.684508,9.349425,-1.339022,2.487551,-0.412411,9.135285,-9.236037,87.458125,9.636491,6.948047,-8.649643,12.380355,-32.207457,7.122436,3.343472,0.520597,6.922486,0.784753,-0.853393,0.241062,0.356818,0.09339,-1.126567,-0.003194,-0.247184,-0.445807,0.044507
1,186851,course1,XBQWgzwe,75.0,course_class1,0,c259,c1730,c93,c14,237,0.004219,0.047619,5,xqbAlQCz,xqbAlQCz,Y7dW6m3f,31.285633,-1.629811,5.932618,-9.677644,7.299567,-4.862344,6.169012,3.902276,3.292172,-0.99198,12.841719,10.642561,-0.6734,-3.290808,4.086778,2.139487,-1.512143,-2.402073,-0.272539,-1.267784,65.90831,10.235587,8.739786,1.621543,7.783871,11.785037,3.029331,11.168388,-9.006072,36.992494,19.882085,-12.459994,31.111172,-4.684508,9.349425,-1.339022,2.487551,-0.412411,9.135285,-9.236037,87.458125,9.636491,6.948047,-8.649643,12.380355,-32.207457,7.122436,3.343472,0.520597,6.922486,0.784753,-0.853393,0.241062,0.356818,0.09339,-1.126567,-0.003194,-0.247184,-0.445807,0.044507
2,478370,course1,XBQWgzwe,86.0,course_class1,0,c259,c1730,c93,c14,237,0.004219,0.047619,5,xqbAlQCz,xqbAlQCz,Y7dW6m3f,31.285633,-1.629811,5.932618,-9.677644,7.299567,-4.862344,6.169012,3.902276,3.292172,-0.99198,12.841719,10.642561,-0.6734,-3.290808,4.086778,2.139487,-1.512143,-2.402073,-0.272539,-1.267784,65.90831,10.235587,8.739786,1.621543,7.783871,11.785037,3.029331,11.168388,-9.006072,36.992494,19.882085,-12.459994,31.111172,-4.684508,9.349425,-1.339022,2.487551,-0.412411,9.135285,-9.236037,87.458125,9.636491,6.948047,-8.649643,12.380355,-32.207457,7.122436,3.343472,0.520597,6.922486,0.784753,-0.853393,0.241062,0.356818,0.09339,-1.126567,-0.003194,-0.247184,-0.445807,0.044507
3,692328,course1,XBQWgzwe,81.0,course_class1,1,c259,c1730,c93,c14,237,0.004219,0.047619,5,xqbAlQCz,xqbAlQCz,Y7dW6m3f,31.285633,-1.629811,5.932618,-9.677644,7.299567,-4.862344,6.169012,3.902276,3.292172,-0.99198,12.841719,10.642561,-0.6734,-3.290808,4.086778,2.139487,-1.512143,-2.402073,-0.272539,-1.267784,65.90831,10.235587,8.739786,1.621543,7.783871,11.785037,3.029331,11.168388,-9.006072,36.992494,19.882085,-12.459994,31.111172,-4.684508,9.349425,-1.339022,2.487551,-0.412411,9.135285,-9.236037,87.458125,9.636491,6.948047,-8.649643,12.380355,-32.207457,7.122436,3.343472,0.520597,6.922486,0.784753,-0.853393,0.241062,0.356818,0.09339,-1.126567,-0.003194,-0.247184,-0.445807,0.044507
4,509128,course1,XBQWgzwe,75.0,course_class1,0,c259,c1730,c93,c14,237,0.004219,0.047619,5,xqbAlQCz,xqbAlQCz,Y7dW6m3f,31.285633,-1.629811,5.932618,-9.677644,7.299567,-4.862344,6.169012,3.902276,3.292172,-0.99198,12.841719,10.642561,-0.6734,-3.290808,4.086778,2.139487,-1.512143,-2.402073,-0.272539,-1.267784,65

In [195]:
tmp_course_exam = course_exam[['course','exam_id','hard']]

In [196]:
df_train = df_train.merge(tmp_course_exam ,on = ['course','exam_id'],how = 'left')

In [197]:
#tmp_rank

In [198]:
get_mean = lambda x: x.sort_values(ascending = False)[1:-1].mean()
get_mean2 = lambda x: x.sort_values(ascending = False)[2:-2].mean()
get_ptp = lambda x: x.sort_values(ascending = False)[2:-2].ptp()
get_std = lambda x: x.sort_values(ascending = False)[2:-2].std()
get_mod = lambda x: pd.Series(data=x).mode().max()
#最后5次的成绩/最后10次的成绩
get_mean_last5 = lambda x: x[-5:].mean()
get_mean_last10 = lambda x: x[-10:].mean()
get_mean_last20 = lambda x: x[-20:].mean()
get_mean_last30 = lambda x: x[-30:].mean()
get_mean_lasthalf = lambda x: x[-int(len(x)/2):].mean()
df_train['per_exam_rank']=df_train.groupby(by=['exam_id'], as_index=False)['score'].rank()
tmp_rank =df_train.groupby(by=['student_id','course'], as_index=False)['per_exam_rank'].agg({'rank_last5_mean':get_mean_last5,'rank_last10_mean':get_mean_last10,'rank_last20_mean':get_mean_last20,'rank_last30_mean':get_mean_last30,'rank_lasthalf_mean':get_mean_lasthalf,'mean_rank':np.mean, 'std_rank':np.std,'max_rank':np.max,'min_rank':np.min,'median_rank':np.median})
df_all=df_all.merge(tmp_rank, on=['student_id','course'], how='left')

In [199]:
# process_mean = lambda x: x[((x >= (np.percentile(x,25) - 1.5*(np.percentile(x,75)-np.percentile(x,25)))) & 
# (x <= (np.percentile(x,75) + 1.5*(np.percentile(x,75)-np.percentile(x,25)))))].mean()
# process_std = lambda x: x[((x >= (np.percentile(x,25) - 1.5*(np.percentile(x,75)-np.percentile(x,25)))) & 
# (x <= (np.percentile(x,75) + 1.5*(np.percentile(x,75)-np.percentile(x,25)))))].std()
# process_index = lambda x: list(x[((x >= (np.percentile(x,25) - 1.5*(np.percentile(x,75)-np.percentile(x,25)))) & 
# (x <= (np.percentile(x,75) + 1.5*(np.percentile(x,75)-np.percentile(x,25)))))].index)

# tmp_process = df_train.groupby(by=['student_id','course'], as_index=False)['score'].agg({'process_index':process_index,'process_mean':process_mean,'process_std':process_std})

In [200]:

# student_id=student['student_id'].values
# def get_lstm(number_list,col):
#     length=len(number_list)
#     #number_list应该是seriers
#     look_back=4
#     #定义初始化pandas
#     feature = pd.DataFrame(columns=['lstm_mean', 'lstm_std','lstm_trend'])
#     for i in range(length):
#         feature_list=[]
#         if i <look_back:
#             feature_list.append(number_list[col[i]])
#             feature_list.append(number_list[col[i]])
#             feature_list.append(1)
#             feature.loc[i]=feature_list
#         else:
#             feature_list.append(np.mean(number_list[col[i-look_back:i]]))
#             feature_list.append(np.std(number_list[col[i-look_back:i]]))
#             for_mean=np.mean(number_list[col[i-2:i]])
#             back_mean=np.mean(number_list[col[i-4:i-2]])
#             feature_list.append(float(for_mean)/float(back_mean))
#             feature.loc[i]=feature_list
#     return feature
# lstm=pd.DataFrame(columns=['score_lstm_mean','score_lstm_std','score_lstm_trend'])
# for j in range(len(student_id)):
#     for k in range(1,9):
#         for i in 
        
#         col=[i for i in df_all.index if df_all.iloc[i]['student_id']==student_id[j] and df_all.iloc[i]['course']=='course'+str(k)]
#         score_list=df_all.iloc[col]['score']
#         #将考试的前几次作为特征
#         lstm.loc[col[i]]
#         score_lstm=get_lstm(score_list,col)#返回pd
#         for i in range(len(col)):
#             lstm.loc[col[i]]=(score_lstm.loc[i].values).tolist()
#         print(j*8+k)
# print(lstm.shape)
# df_all = pd.concat([df_all, lstm], axis=1)
# df_all.to_csv('./lstm_all.csv')       
    #   rank_list=df_all.iloc[col]['per_exam_rank']   
    # rank_lstm=get_lstm(rank_list)
    #     df_all.iloc[col]['rank_lstm_mean']=rank_lstm['lstm_mean']
    #     df_all.iloc[col]['rank_lstm_max']=rank_lstm['lstm_max']
    #     df_all.iloc[col]['rank_lstm_min']=rank_lstm['lstm_min']
    #     df_all.iloc[col]['rank_lstm_median']=rank_lstm['lstm_median']
    #     df_all.iloc[col]['rank_lstm_std']=rank_lstm['lstm_std']
    #     df_all.iloc[col]['rank_lstm_trend']=rank_lstm['lstm_trean']
# index_list = []
# for i in tmp_process['process_index']:
#     index_list = index_list+i

In [201]:
tmp1 = df_train.groupby(by=['student_id','course'], as_index=False)['score'].agg({'lasthalf':get_mean_lasthalf,'last30':get_mean_last30,'last20':get_mean_last20,'last10':get_mean_last10,'last5':get_mean_last5,'mode':get_mod,'mean_score':np.mean, 'median_score':np.median, 'std_score':np.std,'max_score':np.max,'min_score':np.min,'cos_mean_score':get_mean,'cos_mean_score2':get_mean2,'ptp':np.ptp ,'var':np.var})
tmp2 = df_train.groupby(by=['student_id'], as_index=False)['score'].agg({'s_mode':get_mod,'s_mean_score':np.mean, 's_median_score':np.median, 's_std_score':np.std,'s_max_score':np.max,'s_min_score':np.min,'s_cos_mean_score':get_mean,'s_cos_mean_score2':get_mean2,'s_ptp':np.ptp  ,'s_var':np.var})

In [202]:
tmp_hard = df_train.groupby(by=['student_id','course'], as_index=False)['hard'].agg({'hard_lasthalf':get_mean_lasthalf,'hard_last30':get_mean_last30,'hard_last20':get_mean_last20,'hard_last10':get_mean_last10,
'hard_last5':get_mean_last5,'hard_mean':np.mean})

In [203]:
rank_course1=tmp1.loc[tmp1['course']=='course1']['mean_score'].rank()
rank_course2=tmp1.loc[tmp1['course']=='course2']['mean_score'].rank()
rank_course3=tmp1.loc[tmp1['course']=='course3']['mean_score'].rank()
rank_course4=tmp1.loc[tmp1['course']=='course4']['mean_score'].rank()
rank_course5=tmp1.loc[tmp1['course']=='course5']['mean_score'].rank()
rank_course6=tmp1.loc[tmp1['course']=='course6']['mean_score'].rank()
rank_course7=tmp1.loc[tmp1['course']=='course7']['mean_score'].rank()
rank_course8=tmp1.loc[tmp1['course']=='course8']['mean_score'].rank()
dict_course1 = {'student_id':tmp2['student_id'],'course':'course1','rank':rank_course1.values}
df_course1 = pd.DataFrame(dict_course1)
dict_course2 = {'student_id':tmp2['student_id'],'course':'course2','rank':rank_course2.values}
df_course2 = pd.DataFrame(dict_course2)
dict_course3 = {'student_id':tmp2['student_id'],'course':'course3','rank':rank_course3.values}
df_course3= pd.DataFrame(dict_course3)
dict_course4 = {'student_id':tmp2['student_id'],'course':'course4','rank':rank_course4.values}
df_course4 = pd.DataFrame(dict_course4)
dict_course5 = {'student_id':tmp2['student_id'],'course':'course5','rank':rank_course5.values}
df_course5 = pd.DataFrame(dict_course5)
dict_course6 = {'student_id':tmp2['student_id'],'course':'course6','rank':rank_course6.values}
df_course6 = pd.DataFrame(dict_course6)
dict_course7 = {'student_id':tmp2['student_id'],'course':'course7','rank':rank_course7.values}
df_course7 = pd.DataFrame(dict_course7)
dict_course8 = {'student_id':tmp2['student_id'],'course':'course8','rank':rank_course8.values}
df_course8 = pd.DataFrame(dict_course8)
#tmp1=tmp1.merge(rank_course1,on=rank_course1.index, how='left')
df_rank=pd.concat([df_course1,df_course2,df_course3,df_course4,df_course5,df_course6,df_course7,df_course8])
#exam_sum = df_train.groupby(by=['student_id','exam_id'], as_index=False)['score'].agg({'s_sum_score':np.sum})
#exam_rank=exam_sum.groupby(bu=['student_id','exam_id'],as_index=False)['s_sum_score'].agg({'s_rank_sum':np.sum()}))
df_all=df_all.merge(tmp1, on=['student_id','course'], how='left')
df_all=df_all.merge(tmp_hard, on=['student_id','course'], how='left')
#df_all=df_all.merge(tmp_process, on=['student_id','course'], how='left')
df_all=df_all.merge(tmp2, on=['student_id'], how='left')
df_all=df_all.merge(df_rank, on=['student_id','course'], how='left')

In [204]:
df_all.head()

,student_id,course,exam_id,score,course_class,gender,cluster_50,cluster_40,cluster_30,cluster,hard,hard_inverse,ration_know,number_know,sim_exam_id,sectionsim_exam_id,catsim_exam_id,k_feature_1,k_feature_2,k_feature_3,k_feature_4,k_feature_5,k_feature_6,k_feature_7,k_feature_8,k_feature_9,k_feature_10,k_feature_11,k_feature_12,k_feature_13,k_feature_14,k_feature_15,k_feature_16,k_feature_17,k_feature_18,k_feature_19,k_feature_20,S_feature_1,S_feature_2,S_feature_3,S_feature_4,S_feature_5,S_feature_6,S_feature_7,S_feature_8,S_feature_9,S_feature_10,S_feature_11,S_feature_12,S_feature_13,S_feature_14,S_feature_15,S_feature_16,S_feature_17,S_feature_18,S_feature_19,S_feature_20,C_feature_1,C_feature_2,C_feature_3,C_feature_4,C_feature_5,C_feature_6,C_feature_7,C_feature_8,C_feature_9,C_feature_10,C_feature_11,C_feature_12,C_feature_13,C_feature_14,C_feature_15,C_feature_16,C_feature_17,C_feature_18,C_feature_19,C_feature_20,rank_last5_mean,rank_last10_mean,rank_last20_mean,rank_last30_mean,rank_lasthalf_mean,mean_rank,std_rank,max_rank,min_rank,median_rank,lasthalf,last30,last20,last10,last5,mode,mean_score,median_score,std_score,max_score,min_score,cos_mean_score,cos_mean_score2,ptp,var,hard_lasthalf,hard_last30,hard_last20,hard_last10,hard_last5,hard_mean,s_mode,s_mean_score,s_median_score,s_std_score,s_max_score,s_min_score,s_cos_mean_score,s_cos_mean_score2,s_ptp,s_var,rank
0,230748,course1,XBQWgzwe,71.0,course_class1,0,c259,c1730,c93,c14,237,0.004219,0.047619,5,xqbAlQCz,xqbAlQCz,Y7dW6m3f,31.285633,-1.629811,5.932618,-9.677644,7.299567,-4.862344,6.169012,3.902276,3.292172,-0.99198,12.841719,10.642561,-0.6734,-3.290808,4.086778,2.139487,-1.512143,-2.402073,-0.272539,-1.267784,65.90831,10.235587,8.739786,1.621543,7.783871,11.785037,3.029331,11.168388,-9.006072,36.992494,19.882085,-12.459994,31.111172,-4.684508,9.349425,-1.339022,2.487551,-0.412411,9.135285,-9.236037,87.458125,9.636491,6.948047,-8.649643,12.380355,-32.207457,7.122436,3.343472,0.520597,6.922486,0.784753,-0.853393,0.241062,0.356818,0.09339,-1.126567,-0.003194,-0.247184,-0.445807,0.044507,224.0,262.50,215.400,204.016667,188.059524,138.880952,89.127721,416.5,11.5,118.25,81.000000,82.066667,82.60,86.8,84.6,82,77.738095,77.5,6.426825,98,67,77.621951,77.5875,31,41.304073,184.238095,174.666667,169.5,153.7,162.2,199.071429,74,76.078947,76.0,6.064482,98,60,76.069307,76.057947,38,36.777942,113.5
1,186851,course1,XBQWgzwe,75.0,course_class1,0,c259,c1730,c93,c14,237,0.004219,0.047619,5,xqbAlQCz,xqbAlQCz,Y7dW6m3f,31.285633,-1.629811,5.932618,-9.677644,7.299567,-4.862344,6.169012,3.902276,3.292172,-0.99198,12.841719,10.642561,-0.6734,-3.290808,4.086778,2.139487,-1.512143,-2.402073,-0.272539,-1.267784,65.90831,10.235587,8.739786,1.621543,7.783871,11.785037,3.029331,11.168388,-9.006072,36.992494,19.882085,-12.459994,31.111172,-4.684508,9.349425,-1.339022,2.487551,-0.412411,9.135285,-9.236037,87.458125,9.636491,6.948047,-8.649643,12.380355,-32.207457,7.122436,3.343472,0.520597,6.922486,0.784753,-0.853393,0.241062,0.356818,0.09339,-1.126567,-0.003194,-0.247184,-0.445807,0.044507,175.9,135.70,134.550,192.600000,164.523810,164.654762,94.696725,443.5,40.5,134.75,79.642857,81.433333,78.45,79.5,82.0,84,79.107143,78.0,6.109479,98,67,79.024390,78.9500,31,37.325731,184.238095,174.666667,169.5,153.7,162.2,199.071429,84,84.608553,84.0,7.475698,100,67,84.612211,84.615894,33,55.886055,135.0
2,478370,course1,XBQWgzwe,86.0,course_class1,0,c259,c1730,c93,c14,237,0.004219,0.047619,5,xqbAlQCz,xqbAlQCz,Y7dW6m3f,31.285633,-1.629811,5.932618,-9.677644,7.299567,-4.862344,6.169012,3.902276,3.292172,-0.99198,12.841719,10.642561,-0.6734,-3.290808,4.086778,2.139487,-1.512143,-2.402073,-0.272539,-1.267784,65.90831,10.235587,8.739786,1.621543,7.783871,11.785037,3.029331,11.168388,-9.006072,36.992494,19.882085,-12.459994,31.111172,-4.684508,9.349425,-1.339022,2.487551,-0.412411,9.135285,-9.236037,87.458125,9.636491,6.948047,-8.649643,12.380355,-32.207457,7.122436,3.343472,0.520597,6.922486,0.784753,-0.853393

In [205]:
df_all['trend5-10'] = df_all['last5']/df_all['last10']
df_all['trend5-20'] = df_all['last5']/df_all['last20']
df_all['trend5-30'] = df_all['last5']/df_all['last30']
df_all['trend5-half'] = df_all['last5']/df_all['lasthalf']
df_all['trend5-all'] = df_all['last5']/df_all['mean_score']
df_all['trend10-20'] = df_all['last10']/df_all['last20']
df_all['trend10-20'] = df_all['last10']/df_all['last30']
df_all['trend10-half'] = df_all['last10']/df_all['lasthalf']
df_all['trend10-all'] = df_all['last10']/df_all['mean_score']
df_all['trend20-half'] = df_all['last20']/df_all['lasthalf']
df_all['trend10-all'] = df_all['last10']/df_all['mean_score']
df_all['score_gap']=df_all['max_score']-df_all['min_score']

#排名的的变动
df_all['rank5-10']=df_all['rank_last5_mean']/df_all['rank_last10_mean']
df_all['rank5-20']=df_all['rank_last5_mean']/df_all['rank_last20_mean']
df_all['rank5-30']=df_all['rank_last5_mean']/df_all['rank_last30_mean']
df_all['rank5-half']=df_all['rank_last5_mean']/df_all['rank_lasthalf_mean']
df_all['rank5-all']=df_all['rank_last5_mean']/df_all['mean_rank']
df_all['rank10-20']=df_all['rank_last10_mean']/df_all['rank_last20_mean']
df_all['rank10-30']=df_all['rank_last10_mean']/df_all['rank_last30_mean']
df_all['rank10-half']=df_all['rank_last10_mean']/df_all['rank_lasthalf_mean']
df_all['rank10-all']=df_all['rank_last10_mean']/df_all['mean_rank']
df_all['rank20-half']=df_all['rank_last20_mean']/df_all['rank_lasthalf_mean']
df_all['rank20-all']=df_all['rank_last20_mean']/df_all['mean_rank']
df_all['rank_gap']=df_all['max_rank']-df_all['min_rank']


In [206]:
#成绩和难度的占比
df_all['trend5-10_hard'] = (df_all['last5']/df_all['hard_last5'])/(df_all['last10']/df_all['hard_last10'])
df_all['trend5-20_hard'] = (df_all['last5']/df_all['hard_last5'])/(df_all['last20']/df_all['hard_last20'])
df_all['trend5-30_hard'] = (df_all['last5']/df_all['hard_last5'])/(df_all['last30']/df_all['hard_last30'])
df_all['trend5-half_hard'] = (df_all['last5']/df_all['hard_last5'])/(df_all['lasthalf']/df_all['hard_lasthalf'])
df_all['trend5-all_hard'] = (df_all['last5']/df_all['hard_last5'])/(df_all['mean_score']/df_all['hard_mean'])
df_all['trend10-20_hard'] = (df_all['last10']/df_all['hard_last10'])/(df_all['last20']/df_all['hard_last20'])
df_all['trend10-20_hard'] = (df_all['last10']/df_all['hard_last10'])/(df_all['last30']/df_all['hard_last30'])
df_all['trend10-half_hard'] = (df_all['last10']/df_all['hard_last10'])/(df_all['lasthalf']/df_all['hard_lasthalf'])
df_all['trend10-all_hard'] = (df_all['last10']/df_all['hard_last10'])/(df_all['mean_score']/df_all['hard_mean'])
df_all['trend20-half_hard'] = (df_all['last20']/df_all['hard_last20'])/(df_all['lasthalf']/df_all['hard_lasthalf'])
df_all['trend20-all_hard'] = (df_all['last20']/df_all['hard_last20'])/(df_all['mean_score']/df_all['hard_mean'])

#排名和难度的占比
df_all['rank5-10_hard'] = (df_all['rank_last5_mean']/df_all['hard_last5'])/(df_all['rank_last10_mean']/df_all['hard_last10'])
df_all['rank5-20_hard'] = (df_all['rank_last5_mean']/df_all['hard_last5'])/(df_all['rank_last20_mean']/df_all['hard_last20'])
df_all['rank5-30_hard'] = (df_all['rank_last5_mean']/df_all['hard_last5'])/(df_all['rank_last30_mean']/df_all['hard_last30'])
df_all['rank5-half_hard'] = (df_all['rank_last5_mean']/df_all['hard_last5'])/(df_all['rank_lasthalf_mean']/df_all['hard_lasthalf'])
df_all['rank5-all_hard'] = (df_all['rank_last5_mean']/df_all['hard_last5'])/(df_all['mean_rank']/df_all['hard_mean'])
df_all['rank10-20_hard'] = (df_all['rank_last10_mean']/df_all['hard_last10'])/(df_all['rank_last20_mean']/df_all['hard_last20'])
df_all['rank10-20_hard'] = (df_all['rank_last10_mean']/df_all['hard_last10'])/(df_all['rank_last30_mean']/df_all['hard_last30'])
df_all['rank10-half_hard'] = (df_all['rank_last10_mean']/df_all['hard_last10'])/(df_all['rank_lasthalf_mean']/df_all['hard_lasthalf'])
df_all['rank10-all_hard'] = (df_all['rank_last10_mean']/df_all['hard_last10'])/(df_all['mean_rank']/df_all['hard_mean'])
df_all['rank20-half_hard'] = (df_all['rank_last20_mean']/df_all['hard_last20'])/(df_all['rank_lasthalf_mean']/df_all['hard_lasthalf'])
df_all['rank20-all_hard'] = (df_all['rank_last20_mean']/df_all['hard_last20'])/(df_all['mean_rank']/df_all['hard_mean'])
#成绩和排名的占比
df_all['trend5-10_rank'] = (df_all['last5']/df_all['rank_last5_mean'])/(df_all['last10']/df_all['rank_last10_mean'])
df_all['trend5-20_rank'] = (df_all['last5']/df_all['rank_last5_mean'])/(df_all['last20']/df_all['rank_last20_mean'])
df_all['trend5-30_rank'] = (df_all['last5']/df_all['rank_last5_mean'])/(df_all['last30']/df_all['rank_last30_mean'])
df_all['trend5-half_rank'] = (df_all['last5']/df_all['rank_last5_mean'])/(df_all['lasthalf']/df_all['rank_lasthalf_mean'])
df_all['trend5-all_rank'] = (df_all['last5']/df_all['rank_last5_mean'])/(df_all['mean_score']/df_all['mean_rank'])
df_all['trend10-20_rank'] = (df_all['last10']/df_all['rank_last10_mean'])/(df_all['last20']/df_all['rank_last20_mean'])
df_all['trend10-20_rank'] = (df_all['last10']/df_all['rank_last10_mean'])/(df_all['last30']/df_all['rank_last30_mean'])
df_all['trend10-half_rank'] = (df_all['last10']/df_all['rank_last10_mean'])/(df_all['lasthalf']/df_all['rank_lasthalf_mean'])
df_all['trend10-all_rank'] = (df_all['last10']/df_all['rank_last10_mean'])/(df_all['mean_score']/df_all['mean_rank'])
df_all['trend20-half_rank'] = (df_all['last20']/df_all['rank_last20_mean'])/(df_all['lasthalf']/df_all['rank_lasthalf_mean'])
df_all['trend20-all_rank'] = (df_all['last20']/df_all['rank_last20_mean'])/(df_all['mean_score']/df_all['mean_rank'])



In [207]:
#tmp_exam = df_train.groupby(by=['exam_id'], as_index=False)['score'].agg({'exam_mean_score':np.mean, 'exam_median_score':np.median, 'exam_std_score':np.std,'exam_max_score':np.max,'exam_min_score':np.min,'exam_cos_mean_score':get_mean,'exam_cos_mean_score2':get_mean2,'exam_ptp':np.ptp ,'exam_var':np.var})
#df_all=df_all.merge(tmp_exam, on=['exam_id'], how='left')

In [208]:
df_all['std/mean'] = df_all['std_score']/df_all['mean_score']
df_all['s_std/mean'] = df_all['s_std_score']/df_all['s_mean_score']
df_all['rank_std/mean']=df_all['std_rank']/df_all['mean_rank']

In [209]:
df_all.head()

,student_id,course,exam_id,score,course_class,gender,cluster_50,cluster_40,cluster_30,cluster,hard,hard_inverse,ration_know,number_know,sim_exam_id,sectionsim_exam_id,catsim_exam_id,k_feature_1,k_feature_2,k_feature_3,k_feature_4,k_feature_5,k_feature_6,k_feature_7,k_feature_8,k_feature_9,k_feature_10,k_feature_11,k_feature_12,k_feature_13,k_feature_14,k_feature_15,k_feature_16,k_feature_17,k_feature_18,k_feature_19,k_feature_20,S_feature_1,S_feature_2,S_feature_3,S_feature_4,S_feature_5,S_feature_6,S_feature_7,S_feature_8,S_feature_9,S_feature_10,S_feature_11,S_feature_12,S_feature_13,S_feature_14,S_feature_15,S_feature_16,S_feature_17,S_feature_18,S_feature_19,S_feature_20,C_feature_1,C_feature_2,C_feature_3,C_feature_4,C_feature_5,C_feature_6,C_feature_7,C_feature_8,C_feature_9,C_feature_10,C_feature_11,C_feature_12,C_feature_13,C_feature_14,C_feature_15,C_feature_16,C_feature_17,C_feature_18,C_feature_19,C_feature_20,rank_last5_mean,rank_last10_mean,rank_last20_mean,rank_last30_mean,rank_lasthalf_mean,mean_rank,std_rank,max_rank,min_rank,median_rank,lasthalf,last30,last20,last10,last5,mode,mean_score,median_score,std_score,max_score,min_score,cos_mean_score,cos_mean_score2,ptp,var,hard_lasthalf,hard_last30,hard_last20,hard_last10,hard_last5,hard_mean,s_mode,s_mean_score,s_median_score,s_std_score,s_max_score,s_min_score,s_cos_mean_score,s_cos_mean_score2,s_ptp,s_var,rank,trend5-10,trend5-20,trend5-30,trend5-half,trend5-all,trend10-20,trend10-half,trend10-all,trend20-half,score_gap,rank5-10,rank5-20,rank5-30,rank5-half,rank5-all,rank10-20,rank10-30,rank10-half,rank10-all,rank20-half,rank20-all,rank_gap,trend5-10_hard,trend5-20_hard,trend5-30_hard,trend5-half_hard,trend5-all_hard,trend10-20_hard,trend10-half_hard,trend10-all_hard,trend20-half_hard,trend20-all_hard,rank5-10_hard,rank5-20_hard,rank5-30_hard,rank5-half_hard,rank5-all_hard,rank10-20_hard,rank10-half_hard,rank10-all_hard,rank20-half_hard,rank20-all_hard,trend5-10_rank,trend5-20_rank,trend5-30_rank,trend5-half_rank,trend5-all_rank,trend10-20_rank,trend10-half_rank,trend10-all_rank,trend20-half_rank,trend20-all_rank,std/mean,s_std/mean,rank_std/mean
0,230748,course1,XBQWgzwe,71.0,course_class1,0,c259,c1730,c93,c14,237,0.004219,0.047619,5,xqbAlQCz,xqbAlQCz,Y7dW6m3f,31.285633,-1.629811,5.932618,-9.677644,7.299567,-4.862344,6.169012,3.902276,3.292172,-0.99198,12.841719,10.642561,-0.6734,-3.290808,4.086778,2.139487,-1.512143,-2.402073,-0.272539,-1.267784,65.90831,10.235587,8.739786,1.621543,7.783871,11.785037,3.029331,11.168388,-9.006072,36.992494,19.882085,-12.459994,31.111172,-4.684508,9.349425,-1.339022,2.487551,-0.412411,9.135285,-9.236037,87.458125,9.636491,6.948047,-8.649643,12.380355,-32.207457,7.122436,3.343472,0.520597,6.922486,0.784753,-0.853393,0.241062,0.356818,0.09339,-1.126567,-0.003194,-0.247184,-0.445807,0.044507,224.0,262.50,215.400,204.016667,188.059524,138.880952,89.127721,416.5,11.5,118.25,81.000000,82.066667,82.60,86.8,84.6,82,77.738095,77.5,6.426825,98,67,77.621951,77.5875,31,41.304073,184.238095,174.666667,169.5,153.7,162.2,199.071429,74,76.078947,76.0,6.064482,98,60,76.069307,76.057947,38,36.777942,113.5,0.974654,1.024213,1.030869,1.044444,1.088270,1.057677,1.071605,1.116570,1.019753,31,0.853333,1.039926,1.097950,1.191112,1.612892,1.218663,1.286660,1.395835,1.890108,1.145382,1.550969,405.0,0.923578,1.070309,1.110102,1.186353,1.335656,1.201957,1.284518,1.446175,1.108421,1.247916,0.808615,1.086729,1.182338,1.352949,1.979536,1.462177,1.673168,2.448058,1.244973,1.821555,1.142173,0.984891,0.938904,0.876865,0.674732,0.822033,0.767716,0.590744,0.890317,0.685083,0.082673,0.079713,0.641756
1,186851,course1,XBQWgzwe,75.0,course_class1,0,c259,c1730,c93,c14,237,0.004219,0.047619,5,xqbAlQCz,xqbAlQCz,Y7dW6m3f,31.285633,-1.629811,5.932618,-9.677644,7.299567,-4.862344,6.169012,3.902276,3.292172,-0.99198,12.841719,10.642561,-0.6734,-3.290808,4.086778,2.139487,-1.512143,-2.402073,-0.272539,-1.267784,65.90831,10.235587,8.739786,1.621543,7.783871,11.785037,3.029331

In [210]:
#all_know

In [211]:
course_exam.head()

,C:0,C:1,C:10,C:11,C:12,C:13,C:14,C:15,C:16,C:17,C:18,C:19,C:2,C:20,C:21,C:22,C:23,C:24,C:25,C:26,C:27,C:28,C:29,C:3,C:30,C:31,C:32,C:33,C:34,C:4,C:5,C:6,C:7,C:8,C:9,K:0,K:1,K:10,K:100,K:101,K:102,K:103,K:104,K:105,K:106,K:107,K:108,K:109,K:11,K:110,K:111,K:112,K:113,K:114,K:115,K:116,K:117,K:118,K:119,K:12,K:120,K:121,K:122,K:123,K:124,K:125,K:126,K:127,K:128,K:129,K:13,K:130,K:131,K:132,K:133,K:134,K:135,K:136,K:137,K:138,K:139,K:14,K:140,K:141,K:142,K:143,K:144,K:145,K:146,K:147,K:148,K:149,K:15,K:150,K:151,K:152,K:153,K:154,K:155,K:156,K:157,K:158,K:159,K:16,K:160,K:161,K:162,K:163,K:164,K:165,K:166,K:167,K:168,K:169,K:17,K:170,K:171,K:172,K:173,K:174,K:175,K:176,K:177,K:178,K:179,K:18,K:180,K:181,K:182,K:183,K:184,K:185,K:186,K:187,K:188,K:189,K:19,K:190,K:191,K:192,K:193,K:194,K:195,K:196,K:197,K:198,K:199,K:2,K:20,K:200,K:201,K:202,K:203,K:204,K:205,K:206,K:207,K:208,K:209,K:21,K:210,K:211,K:212,K:213,K:214,K:215,K:216,K:217,K:218,K:219,K:22,K:220,K:221,K:222,K:223,K:224,K:225,K:226,K:227,K:228,K:229,K:23,K:230,K:231,K:232,K:233,K:234,K:235,K:236,K:237,K:238,K:239,K:24,K:240,K:241,K:242,K:243,K:244,K:245,K:246,K:247,K:248,K:249,K:25,K:250,K:251,K:252,K:253,K:254,K:255,K:256,K:257,K:258,K:259,K:26,K:260,K:261,K:262,K:263,K:264,K:265,K:266,K:267,K:268,K:269,K:27,K:270,K:271,K:272,K:273,K:274,K:275,K:276,K:277,K:278,K:279,K:28,K:280,K:281,K:282,K:283,K:284,K:285,K:286,K:287,K:288,K:289,K:29,K:290,K:291,K:292,K:293,K:294,K:295,K:296,K:297,K:298,K:299,K:3,K:30,K:300,K:301,K:302,K:303,K:304,K:305,K:306,K:307,K:308,K:309,K:31,K:310,K:311,K:312,K:313,K:314,K:315,K:316,K:317,K:318,K:319,K:32,K:320,K:321,K:322,K:323,K:324,K:325,K:326,K:327,K:328,K:329,K:33,K:330,K:331,K:332,K:333,K:334,K:335,K:336,K:337,K:338,K:339,K:34,K:340,K:341,K:342,K:343,K:344,K:345,K:346,K:347,K:348,K:349,K:35,K:350,K:351,K:352,K:353,K:354,K:355,K:356,K:357,K:358,K:359,K:36,K:360,K:361,K:362,K:363,K:364,K:365,K:366,K:367,K:37,K:38,K:39,K:4,K:40,K:41,K:42,K:43,K:44,K:45,K:46,K:47,K:48,K:49,K:5,K:50,K:51,K:52,K:53,K:54,K:55,K:56,K:57,K:58,K:59,K:6,K:60,K:61,K:62,K:63,K:64,K:65,K:66,K:67,K:68,K:69,K:7,K:70,K:71,K:72,K:73,K:74,K:75,K:76,K:77,K:78,K:79,K:8,K:80,K:81,K:82,K:83,K:84,K:85,K:86,K:87,K:88,K:89,K:9,K:90,K:91,K:92,K:93,K:94,K:95,K:96,K:97,K:98,K:99,S:0,S:1,S:10,S:11,S:12,S:13,S:14,S:15,S:16,S:17,S:18,S:19,S:2,S:20,S:21,S:22,S:23,S:24,S:25,S:26,S:27,S:28,S:29,S:3,S:30,S:31,S:32,S:33,S:34,S:35,S:36,S:37,S:38,S:39,S:4,S:40,S:41,S:42,S:43,S:44,S:45,S:46,S:47,S:48,S:49,S:5,S:50,S:51,S:52,S:53,S:54,S:55,S:56,S:57,S:58,S:59,S:6,S:60,S:61,S:62,S:63,S:64,S:65,S:66,S:67,S:68,S:69,S:7,S:70,S:71,S:72,S:73,S:74,S:75,S:76,S:77,S:8,S:9,catsim_exam_id,cluster,cluster_30,cluster_40,cluster_50,course,exam_id,hard,hard_inverse,know_mean,number_know,ration_know,sectionsim_exam_id,sim_exam_id,k_feature_1,k_feature_2,k_feature_3,k_feature_4,k_feature_5,k_feature_6,k_feature_7,k_feature_8,k_feature_9,k_feature_10,k_feature_11,k_feature_12,k_feature_13,k_feature_14,k_feature_15,k_feature_16,k_feature_17,k_feature_18,k_feature_19,k_feature_20,C_feature_1,C_feature_2,C_feature_3,C_feature_4,C_feature_5,C_feature_6,C_feature_7,C_feature_8,C_feature_9,C_feature_10,C_feature_11,C_feature_12,C_feature_13,C_feature_14,C_feature_15,C_feature_16,C_feature_17,C_feature_18,C_feature_19,C_feature_20,S_feature_1,S_feature_2,S_feature_3,S_feature_4,S_feature_5,S_feature_6,S_feature_7,S_feature_8,S_feature_9,S_feature_10,S_feature_11,S_feature_12,S_feature_13,S_feature_14,S_feature_15,S_feature_16,S_feature_17,S_feature_18,S_feature_19,S_feature_20
0,100,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,14,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [212]:
#sub_course_exam = course_exam[['cluster_50','cluster_40','cluster_30','cluster','course','exam_id','hard','hard_inverse','ration_know','number_know','sim_exam_id','sectionsim_exam_id','catsim_exam_id']]

In [213]:
df_all.shape

(307899, 174)

In [214]:
#df_all=df_all.merge(sub_course_exam, on=['exam_id','course'], how='left')
#df_all=df_all.merge(course_exam, on=['exam_id','course'], how='left')

In [215]:
df_all.shape

(307899, 174)

In [216]:
df_all.head()

,student_id,course,exam_id,score,course_class,gender,cluster_50,cluster_40,cluster_30,cluster,hard,hard_inverse,ration_know,number_know,sim_exam_id,sectionsim_exam_id,catsim_exam_id,k_feature_1,k_feature_2,k_feature_3,k_feature_4,k_feature_5,k_feature_6,k_feature_7,k_feature_8,k_feature_9,k_feature_10,k_feature_11,k_feature_12,k_feature_13,k_feature_14,k_feature_15,k_feature_16,k_feature_17,k_feature_18,k_feature_19,k_feature_20,S_feature_1,S_feature_2,S_feature_3,S_feature_4,S_feature_5,S_feature_6,S_feature_7,S_feature_8,S_feature_9,S_feature_10,S_feature_11,S_feature_12,S_feature_13,S_feature_14,S_feature_15,S_feature_16,S_feature_17,S_feature_18,S_feature_19,S_feature_20,C_feature_1,C_feature_2,C_feature_3,C_feature_4,C_feature_5,C_feature_6,C_feature_7,C_feature_8,C_feature_9,C_feature_10,C_feature_11,C_feature_12,C_feature_13,C_feature_14,C_feature_15,C_feature_16,C_feature_17,C_feature_18,C_feature_19,C_feature_20,rank_last5_mean,rank_last10_mean,rank_last20_mean,rank_last30_mean,rank_lasthalf_mean,mean_rank,std_rank,max_rank,min_rank,median_rank,lasthalf,last30,last20,last10,last5,mode,mean_score,median_score,std_score,max_score,min_score,cos_mean_score,cos_mean_score2,ptp,var,hard_lasthalf,hard_last30,hard_last20,hard_last10,hard_last5,hard_mean,s_mode,s_mean_score,s_median_score,s_std_score,s_max_score,s_min_score,s_cos_mean_score,s_cos_mean_score2,s_ptp,s_var,rank,trend5-10,trend5-20,trend5-30,trend5-half,trend5-all,trend10-20,trend10-half,trend10-all,trend20-half,score_gap,rank5-10,rank5-20,rank5-30,rank5-half,rank5-all,rank10-20,rank10-30,rank10-half,rank10-all,rank20-half,rank20-all,rank_gap,trend5-10_hard,trend5-20_hard,trend5-30_hard,trend5-half_hard,trend5-all_hard,trend10-20_hard,trend10-half_hard,trend10-all_hard,trend20-half_hard,trend20-all_hard,rank5-10_hard,rank5-20_hard,rank5-30_hard,rank5-half_hard,rank5-all_hard,rank10-20_hard,rank10-half_hard,rank10-all_hard,rank20-half_hard,rank20-all_hard,trend5-10_rank,trend5-20_rank,trend5-30_rank,trend5-half_rank,trend5-all_rank,trend10-20_rank,trend10-half_rank,trend10-all_rank,trend20-half_rank,trend20-all_rank,std/mean,s_std/mean,rank_std/mean
0,230748,course1,XBQWgzwe,71.0,course_class1,0,c259,c1730,c93,c14,237,0.004219,0.047619,5,xqbAlQCz,xqbAlQCz,Y7dW6m3f,31.285633,-1.629811,5.932618,-9.677644,7.299567,-4.862344,6.169012,3.902276,3.292172,-0.99198,12.841719,10.642561,-0.6734,-3.290808,4.086778,2.139487,-1.512143,-2.402073,-0.272539,-1.267784,65.90831,10.235587,8.739786,1.621543,7.783871,11.785037,3.029331,11.168388,-9.006072,36.992494,19.882085,-12.459994,31.111172,-4.684508,9.349425,-1.339022,2.487551,-0.412411,9.135285,-9.236037,87.458125,9.636491,6.948047,-8.649643,12.380355,-32.207457,7.122436,3.343472,0.520597,6.922486,0.784753,-0.853393,0.241062,0.356818,0.09339,-1.126567,-0.003194,-0.247184,-0.445807,0.044507,224.0,262.50,215.400,204.016667,188.059524,138.880952,89.127721,416.5,11.5,118.25,81.000000,82.066667,82.60,86.8,84.6,82,77.738095,77.5,6.426825,98,67,77.621951,77.5875,31,41.304073,184.238095,174.666667,169.5,153.7,162.2,199.071429,74,76.078947,76.0,6.064482,98,60,76.069307,76.057947,38,36.777942,113.5,0.974654,1.024213,1.030869,1.044444,1.088270,1.057677,1.071605,1.116570,1.019753,31,0.853333,1.039926,1.097950,1.191112,1.612892,1.218663,1.286660,1.395835,1.890108,1.145382,1.550969,405.0,0.923578,1.070309,1.110102,1.186353,1.335656,1.201957,1.284518,1.446175,1.108421,1.247916,0.808615,1.086729,1.182338,1.352949,1.979536,1.462177,1.673168,2.448058,1.244973,1.821555,1.142173,0.984891,0.938904,0.876865,0.674732,0.822033,0.767716,0.590744,0.890317,0.685083,0.082673,0.079713,0.641756
1,186851,course1,XBQWgzwe,75.0,course_class1,0,c259,c1730,c93,c14,237,0.004219,0.047619,5,xqbAlQCz,xqbAlQCz,Y7dW6m3f,31.285633,-1.629811,5.932618,-9.677644,7.299567,-4.862344,6.169012,3.902276,3.292172,-0.99198,12.841719,10.642561,-0.6734,-3.290808,4.086778,2.139487,-1.512143,-2.402073,-0.272539,-1.267784,65.90831,10.235587,8.739786,1.621543,7.783871,11.785037,3.029331

In [217]:
for i in ['cluster_50','cluster_40','cluster_30','cluster','course','course_class','exam_id','student_id','sim_exam_id','sectionsim_exam_id','catsim_exam_id']:
    lbl = LabelEncoder()
    #all_data[i+"_count"] = all_data.groupby([i])[i].transform('count')
    #all_data[i+"_rank"] = all_data[i+"_count"].rank(method='min')
    df_all[i] = lbl.fit_transform(df_all[i].astype(str))

In [218]:
cat_list = ['cluster_50','cluster_40','cluster_30','cluster','course','course_class','exam_id','student_id','sim_exam_id','sectionsim_exam_id','catsim_exam_id']
#cat_list.extend(['k_feature_1','k_feature_2','k_feature_3','k_feature_4','k_feature_5','k_feature_6','k_feature_7','k_feature_8','k_feature_9','k_feature_10'])


In [219]:
print(len(df_train))
print(len(df_test))
print(len(df_all))

303899
4000
307899


In [220]:
df_train = df_all[:len(df_train)]
df_test = df_all[len(df_train):].reset_index(drop=True)
#df_train = df_all[:281352]
#df_test = df_all[281352:].reset_index(drop=True)
col = [i for i in df_all.columns if i not in ['score']]
#X_train = df_train.drop(columns=['score'])
X_train = df_train[col]
y = df_train['score']

In [221]:
feature_name = list(X_train.columns)

In [222]:
sample_weight = df_train['score']/100

In [223]:
cat_col_index = [feature_name.index(value) for value in cat_list]

In [224]:
test_y = np.zeros(len(df_test))
random_seed = 20190806
cv_model = []
cv_score = []
skf = StratifiedKFold(n_splits=3, random_state=random_seed, shuffle=True)
for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
    #print(index)
    train_x, val_x, train_y, val_y = X_train.iloc[train_index], X_train.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    #sw = sample_weight[train_index]
    
    cbt_model = cbt.CatBoostRegressor(iterations=8000,learning_rate=0.05,max_depth=11,
                                       l2_leaf_reg=10,verbose=200,#early_stopping_rounds=250,
                                       eval_metric='RMSE',task_type='CPU')
    cbt_model.fit(train_x, train_y,eval_set=(val_x,val_y),cat_features=cat_col_index,early_stopping_rounds=250)
    
    cv_model.append(cbt_model)
    #lgb.n_estimators = lgb.best_iteration_
    val_y_pred = cbt_model.predict(val_x)
    cv_score.append( np.sqrt(mean_squared_error(val_y,val_y_pred)))
    test_y += cbt_model.predict(df_test[col])/3
print("CV score: ",np.mean(cv_score))

sub_test = pd.read_csv('/home/kesci/input/smart_edu7557/submission_s2.csv')
sub_test['pred'] = test_y
sub_test.to_csv('./result_new.csv',index=None)

0:	learn: 79.7120308	test: 79.7115315	best: 79.7115315 (0)	total: 1.56s	remaining: 3h 27m 30s
200:	learn: 4.2056781	test: 4.2473570	best: 4.2473570 (200)	total: 4m 56s	remaining: 3h 11m 40s
400:	learn: 3.9382108	test: 4.0451340	best: 4.0451340 (400)	total: 9m 56s	remaining: 3h 8m 28s
600:	learn: 3.7683752	test: 3.9777539	best: 3.9777539 (600)	total: 14m 55s	remaining: 3h 3m 41s
800:	learn: 3.6160649	test: 3.9322987	best: 3.9322987 (800)	total: 19m 55s	remaining: 2h 59m 1s
1000:	learn: 3.4790734	test: 3.8978529	best: 3.8978529 (1000)	total: 24m 57s	remaining: 2h 54m 29s
1200:	learn: 3.3561429	test: 3.8723360	best: 3.8723360 (1200)	total: 29m 59s	remaining: 2h 49m 45s
1400:	learn: 3.2428893	test: 3.8492488	best: 3.8492488 (1400)	total: 35m 1s	remaining: 2h 45m
1600:	learn: 3.1349294	test: 3.8315312	best: 3.8315312 (1600)	total: 40m 5s	remaining: 2h 40m 12s
1800:	learn: 3.0328047	test: 3.8163253	best: 3.8163253 (1800)	total: 45m 7s	remaining: 2h 35m 18s
2000:	learn: 2.9371915	test: 3.8036

In [225]:
# test_y = np.zeros(len(df_test))
# random_seed = 20190806
# cv_model = []
# cv_score = []
# skf = StratifiedKFold(n_splits=5, random_state=random_seed, shuffle=True)
# for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
#     #print(index)
#     train_x, val_x, train_y, val_y = X_train.iloc[train_index], X_train.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
#     #sw = sample_weight[train_index]
#     lgb_params = {
#         'boosting_type': 'gbdt',
#         'objective': 'regression',
#         'n_estimators': 10000,
#         #'metric': 'mae',
#         'learning_rate': 0.01,
#         'min_child_samples': 46,
#         'min_child_weight': 0.01,
#         'subsample_freq': 1,
#         'num_leaves': 40,
#         'max_depth': 7,
#         'subsample': 0.42,
#         'colsample_bytree': 0.48,
#         'reg_alpha': 0.15,
#         'reg_lambda': 5,
#         'verbose': -1,
#         'seed': random_seed
#     }
#     lgb = LGBMRegressor(**lgb_params)


#     lgb.fit(
#         train_x,
#         train_y,
#         eval_set=[(train_x, train_y), (val_x, val_y)],
#         #sample_weight = sw,
#         eval_names=['train', 'val'],
#         eval_metric='rmse',
#         #eval_metric='mae',
#         early_stopping_rounds=100,
#         categorical_feature= cat_list,
#         verbose=500,
#     )
#     cv_model.append(lgb)
#     lgb.n_estimators = lgb.best_iteration_
#     val_y_pred = lgb.predict(val_x)
#     cv_score.append( np.sqrt(mean_squared_error(val_y,val_y_pred)))
#     test_y += lgb.predict(df_test[col])/5
# print("CV score: ",np.mean(cv_score))

In [226]:
test_y

array([83.61417189, 77.88933973, 81.9045245 , ..., 74.87039822,
       73.95854251, 72.74876054])

In [227]:
tmp = test_y.copy()

In [228]:
sub_test = pd.read_csv('/home/kesci/input/smart_edu7557/submission_s2.csv')

In [229]:
sub_test['pred'] = test_y

In [230]:
sub_test.head()

,student_id,course,exam_id,pred
0,230748,course1,PcVABZEp,83.614172
1,186851,course1,PcVABZEp,77.889340
2,478370,course1,PcVABZEp,81.904525
3,692328,course1,PcVABZEp,79.179295
4,509128,course1,PcVABZEp,77.926492


In [231]:
sub_test.to_csv('./result_new.csv',index=None)

In [232]:
fi = []
for i in cv_model:
    tmp = {
        'name' : col,
        'score' : i.feature_importances_
    }
    fi.append(pd.DataFrame(tmp))
    
fi = pd.concat(fi)
fig = plt.figure(figsize=(8,8))
fi.groupby(['name'])['score'].agg('mean').sort_values(ascending=False).head(40).plot.barh()

<Figure size 576x576 with 1 Axes>

In [233]:
#!wget -O tianyi_submit https://cdn.kesci.com/submit_tool/v3/tianyi_submit&&chmod +x tianyi_submit

In [234]:
!./tianyi_submit -file result_new.csv -token 1fa3946c75541f30

Tianyi Submit Tool 3.2.1

> 已验证Token
> 提交文件 result_new.csv (163.51 KiB)
> 已上传 100 %
> 文件已上传        
> 服务器响应: 200 提交成功，请等待评审完成
> 提交完成
